# Assignment 1 - Investigating Transformer Models

Course: Deep Learning for NLP

This notebook is organized by the assignment rubric:
1. How models read text (BERT tokenizer)
2. Generating language (GPT)
3. BERT vs GPT behavior
4. Tokenizer/model mismatch
5. Summarization

In [ ]:
import random
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    AutoTokenizer,
    pipeline,
    set_seed,
)

random.seed(42)
torch.manual_seed(42)

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

DEVICE = 0 if torch.cuda.is_available() else -1
print("CUDA available:", torch.cuda.is_available())

## Part 1 - How Models Read Text (BERT tokenizer)

Goal: Investigate how sentence properties (capitalization, punctuation, numbers, rare words, length) affect tokenization.

In [ ]:
bert_tok = AutoTokenizer.from_pretrained("bert-base-uncased")

sentences = [
    "Hello world!",
    "HELLO world!",
    "Hello, world!!!",
    "The number 1024 is important.",
    "Supercalifragilisticexpialidocious is uncommon.",
    "A very very very very very long sentence with repeated words.",
]

rows = []
for s in sentences:
    ids = bert_tok(s, add_special_tokens=True)["input_ids"]
    toks = bert_tok.convert_ids_to_tokens(ids)
    rows.append({
        "text": s,
        "char_len": len(s),
        "token_len": len(toks),
        "tokens": " ".join(toks),
    })

sentence_df = pd.DataFrame(rows)
sentence_df

In [ ]:
plt.figure(figsize=(7, 4))
plt.scatter(sentence_df["char_len"], sentence_df["token_len"], s=70)
plt.xlabel("Character length")
plt.ylabel("BERT token count")
plt.title("Part 1: Character vs Token Length")
plt.tight_layout()
plt.show()

sentence_df.to_csv(OUTPUT_DIR / "part1_sentence_tokenization.csv", index=False)

In [ ]:
tips = sns.load_dataset("tips").copy()

def row_to_text(r):
    return (
        f"Party size {int(r['size'])} paid total bill ${r['total_bill']:.2f} with tip ${r['tip']:.2f}. "
        f"Smoker={r['smoker']}, Day={r['day']}, Time={r['time']}, Sex={r['sex']}."
    )

tips["description"] = tips.apply(row_to_text, axis=1)
tips["char_len"] = tips["description"].str.len()
tips["token_len"] = tips["description"].apply(lambda x: len(bert_tok(x)["input_ids"]))

plt.figure(figsize=(7, 4))
sns.histplot(tips["token_len"], bins=20)
plt.xlabel("BERT token count")
plt.ylabel("Rows")
plt.title("Part 1: Token length distribution for tips rows")
plt.tight_layout()
plt.show()

tips[["description", "char_len", "token_len"]].head()

### Part 1 observations
- Numbers and punctuation often split into separate tokens; long rare words split into subwords.
- Character length and token length are correlated but not identical.
- Token count varies because tokenizers encode frequent chunks efficiently and break unfamiliar strings into more pieces.

## Part 2 - Generating Language (GPT)

Goal: Compare stability vs creativity by changing decoding settings.

In [ ]:
gpt_tok = AutoTokenizer.from_pretrained("distilgpt2")
gpt_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
gpt_model.eval()

if gpt_tok.pad_token_id is None:
    gpt_tok.pad_token = gpt_tok.eos_token

prompts = [
    "The capital of France is",
    "I opened the fridge and found",
    "The number 1024 is important because",
]

sample = tips.sample(1, random_state=7).iloc[0]
prompts.append(
    "Restaurant record: "
    f"bill ${sample.total_bill:.2f}, tip ${sample.tip:.2f}, day {sample.day}, "
    f"time {sample.time}, party size {int(sample.size)}."
)

def generate_many(prompt, runs, **kwargs):
    out = []
    for i in range(runs):
        set_seed(100 + i)
        ids = gpt_tok(prompt, return_tensors="pt").input_ids
        with torch.no_grad():
            gen = gpt_model.generate(ids, **kwargs)
        out.append(gpt_tok.decode(gen[0], skip_special_tokens=True))
    return out

greedy_cfg = dict(max_new_tokens=35, do_sample=False, pad_token_id=gpt_tok.eos_token_id)
creative_cfg = dict(max_new_tokens=35, do_sample=True, temperature=0.9, top_p=0.9, pad_token_id=gpt_tok.eos_token_id)

rows = []
for p in prompts:
    for i, text in enumerate(generate_many(p, 3, **greedy_cfg), start=1):
        rows.append({"prompt": p, "setting": "greedy", "run": i, "output": text})
    for i, text in enumerate(generate_many(p, 3, **creative_cfg), start=1):
        rows.append({"prompt": p, "setting": "creative", "run": i, "output": text})

gen_df = pd.DataFrame(rows)
gen_df.head(12)

In [ ]:
gen_df.to_csv(OUTPUT_DIR / "part2_gpt_generations.csv", index=False)

comparison = (
    gen_df.groupby(["prompt", "setting"])
    .size()
    .reset_index(name="n_outputs")
)
comparison

### Part 2 observations
- Greedy decoding is more stable and often repeats high-probability phrasing.
- Sampling (`temperature`, `top_p`) increases diversity and can introduce instability.
- Creativity is controlled mainly by decoding strategy and sampling parameters.

## Part 3 - When Models Refuse to Behave (BERT vs GPT)

Test sentence: `Deep learning models are very`

In [ ]:
prompt = "Deep learning models are very"

ids = gpt_tok(prompt, return_tensors="pt").input_ids
with torch.no_grad():
    gpt_out = gpt_model.generate(
        ids,
        max_new_tokens=20,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        pad_token_id=gpt_tok.eos_token_id,
    )

gpt_result = gpt_tok.decode(gpt_out[0], skip_special_tokens=True)

fill_mask = pipeline("fill-mask", model="bert-base-uncased", tokenizer="bert-base-uncased", device=DEVICE)
masked = "Deep learning models are very [MASK]."
bert_preds = fill_mask(masked, top_k=5)

compare_rows = [{
    "model": "GPT (distilgpt2)",
    "task": "continuation",
    "input": prompt,
    "result": gpt_result,
}]

for pred in bert_preds:
    compare_rows.append({
        "model": "BERT (bert-base-uncased)",
        "task": "fill-mask",
        "input": masked,
        "result": f"{pred['token_str'].strip()} (score={pred['score']:.4f})",
    })

compare_df = pd.DataFrame(compare_rows)
compare_df

### Part 3 explanation
- GPT succeeds at continuation because it predicts the next token left-to-right.
- BERT fails at open-ended continuation because it is not trained as an autoregressive generator.
- BERT works differently by predicting a masked token using both left and right context.

## Part 3.5 - Can We Swap Their Brains? (Tokenizer vs Model)

In [ ]:
mismatch_rows = []

# GPT model + BERT tokenizer
try:
    bert_tok2 = AutoTokenizer.from_pretrained("bert-base-uncased")
    gpt_model2 = AutoModelForCausalLM.from_pretrained("distilgpt2")
    ids = bert_tok2("The capital of France is", return_tensors="pt").input_ids
    with torch.no_grad():
        out = gpt_model2.generate(ids, max_new_tokens=12)
    mismatch_rows.append({
        "pair": "GPT model + BERT tokenizer",
        "status": "ran",
        "outcome": bert_tok2.decode(out[0], skip_special_tokens=True),
    })
except Exception as e:
    mismatch_rows.append({
        "pair": "GPT model + BERT tokenizer",
        "status": "error",
        "outcome": str(e),
    })

# BERT model + GPT tokenizer
try:
    gpt_tok2 = AutoTokenizer.from_pretrained("distilgpt2")
    bert_mlm = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
    ids = gpt_tok2("Deep learning models are very", return_tensors="pt").input_ids
    with torch.no_grad():
        logits = bert_mlm(ids).logits
    pred_id = int(torch.argmax(logits[0, -1]).item())
    mismatch_rows.append({
        "pair": "BERT model + GPT tokenizer",
        "status": "ran",
        "outcome": f"Predicted id {pred_id} decoded by GPT tokenizer as: {gpt_tok2.decode([pred_id])}",
    })
except Exception as e:
    mismatch_rows.append({
        "pair": "BERT model + GPT tokenizer",
        "status": "error",
        "outcome": str(e),
    })

mismatch_df = pd.DataFrame(mismatch_rows)
mismatch_df

### Final question answer
A pretrained model generally cannot be paired with a different tokenizer because token IDs are part of the model's learned representation. If token ID 1234 meant one subword during training but maps to a different token at inference time, the embedding lookup and all downstream layers receive semantically incorrect inputs, causing degraded or nonsensical behavior.

## Part 4 - Compressing Information (Summarization)

In [ ]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=DEVICE)

natural_paragraph = (
    "Transformer models are used for classification, question answering, and generation. "
    "They perform strongly but can still hallucinate facts or repeat phrases under some decoding settings. "
    "Careful evaluation is needed before deployment in high-stakes use cases."
)

penguins = sns.load_dataset("penguins").dropna().head(8)
structured_text = " ".join([
    f"Penguin {i}: species {r.species}, island {r.island}, bill length {r.bill_length_mm:.1f} mm, "
    f"bill depth {r.bill_depth_mm:.1f} mm, flipper {r.flipper_length_mm:.1f} mm, mass {r.body_mass_g:.1f} g."
    for i, r in penguins.iterrows()
])

mixed_text = (
    "Field notes from a biology class: students measured penguins and discussed species differences. "
    + structured_text +
    " The instructor asked students to interpret size patterns across islands."
)

inputs = {
    "natural_paragraph": natural_paragraph,
    "structured_penguins": structured_text,
    "mixed_structured_natural": mixed_text,
}

sum_rows = []
for name, text in inputs.items():
    out = summarizer(text, max_length=75, min_length=20, do_sample=False)[0]["summary_text"]
    sum_rows.append({"input_name": name, "summary": out})

sum_df = pd.DataFrame(sum_rows)
sum_df

### Part 4 observations
- One summary that works well: `natural_paragraph` usually preserves the main message.
- One summary that loses important information: `structured_penguins` often drops numeric detail.
- Consistent mistake: model compresses many measurements into vague language.
- Structured data is difficult because summarization models are mainly trained on narrative text, not dense tabular facts.

## Submission checklist
- Experiments included for all required parts.
- Tables/figures included for tokenization and generation comparison.
- Short explanations provided under each section.

Run all cells before submission and verify outputs are visible.